<a href="https://colab.research.google.com/github/Ujjwalb2/Rnn-model-chatbot/blob/main/Copy_of_multitext_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

True

In [ ]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score



In [ ]:
np.random.seed(500)

In [ ]:
data_1 = pd.read_csv(r"train.csv",encoding='latin-1')

In [ ]:
def data_processing(data):
  data['text'].dropna(inplace=True)
  data['text'] = [entry.lower() for entry in data['text']]
  data['text']= [word_tokenize(entry) for entry in data['text']]
  word_Lemmatized = WordNetLemmatizer()
  for index, entry in enumerate(data['text']):
    # print(index, entry)
    # print("--------------------------------")
    Final_words = []
    for word, tag in pos_tag(entry):
        # print(word, tag)
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,pos = tag_map[tag[0]])
            Final_words.append(word_Final)
    data.loc[index,'text_final'] = str(Final_words)

  return data


In [ ]:
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV

In [ ]:
data_1=data_processing(data_1)

In [ ]:

data_1

,category,text,text_final
0,tech,"[tv, future, in, the, hands, of, viewers, with...","['tv', 'future', 'hand', 'viewer', 'home', 'th..."
1,business,"[worldcom, boss, left, books, alone, former, w...","['worldcom', 'bos', 'leave', 'book', 'alone', ..."
2,sport,"[tigers, wary, of, farrell, gamble, leicester,...","['tiger', 'wary', 'farrell', 'gamble', 'leices..."
3,sport,"[yeading, face, newcastle, in, fa, cup, premie...","['yeading', 'face', 'newcastle', 'fa', 'cup', ..."
4,entertainment,"[ocean, s, twelve, raids, box, office, ocean, ...","['ocean', 'twelve', 'raid', 'box', 'office', '..."
...,...,...,...
2220,business,"[cars, pull, down, us, retail, figures, us, re...","['car', 'pull', 'u', 'retail', 'figure', 'u', ..."
2221,politics,"[kilroy, unveils, immigration, policy, ex-chat...","['kilroy', 'unveils', 'immigration', 'policy',..."
2222,entertainment,"[rem, announce, new, glasgow, concert, us, ban...","['rem', 'announce', 'new', 'glasgow', 'concert..."
2223,politics,"[how, political, squabbles, snowball, it, s, b...","['political', 'squabble', 'snowball', 'become'..."


In [ ]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(data_1['text_final'],data_1['category'],test_size=0.3)

In [ ]:
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)

In [ ]:
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(data_1['text_final'])

TfidfVectorizer(max_features=5000)

In [ ]:
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

model 1


In [ ]:
NB = naive_bayes.MultinomialNB()
NB.fit(Train_X_Tfidf,Train_Y)


MultinomialNB()

In [ ]:
# predict the labels on validation dataset
pred_NB = NB.predict(Test_X_Tfidf)

# Use accuracy_score function to get the accuracy
print("Naive Bayes Accuracy Score -> ",accuracy_score(pred_NB, Test_Y)*100)

Naive Bayes Accuracy Score ->  97.30538922155688


model 2

In [ ]:
SVM = svm.SVC(C = 1.0, kernel = 'linear', degree = 3, gamma = 'auto')
SVM.fit(Train_X_Tfidf, Train_Y)

SVC(gamma='auto', kernel='linear')

In [ ]:
pred_SVM = SVM.predict(Test_X_Tfidf)

In [ ]:
print("SVM Accuracy Score -> ",accuracy_score(pred_SVM, Test_Y)*100)

SVM Accuracy Score ->  98.35329341317365


## **our model is ready now , we just need to import data now with colums category and text.**

assuming data as train_new


In [ ]:
data_test = pd.read_csv(r"train_new.csv",encoding='latin-1')

In [ ]:
data_test=data_processing(data_test)

In [ ]:
data_test

,category,text,text_final
0,tech,"[tv, future, in, the, hands, of, viewers, with...","['tv', 'future', 'hand', 'viewer', 'home', 'th..."
1,business,"[worldcom, boss, left, books, alone, former, w...","['worldcom', 'bos', 'leave', 'book', 'alone', ..."
2,politics,"[howard, hits, back, at, mongrel, jibe, michae...","['howard', 'hit', 'back', 'mongrel', 'jibe', '..."
3,politics,"[blair, prepares, to, name, poll, date, tony, ...","['blair', 'prepare', 'name', 'poll', 'date', '..."
4,entertainment,"[last, star, wars, not, for, children, the, si...","['last', 'star', 'war', 'child', 'sixth', 'fin..."
...,...,...,...
2214,business,"[cars, pull, down, us, retail, figures, us, re...","['car', 'pull', 'u', 'retail', 'figure', 'u', ..."
2215,politics,"[kilroy, unveils, immigration, policy, ex-chat...","['kilroy', 'unveils', 'immigration', 'policy',..."
2216,entertainment,"[rem, announce, new, glasgow, concert, us, ban...","['rem', 'announce', 'new', 'glasgow', 'concert..."
2217,politics,"[how, political, squabbles, snowball, it, s, b...","['political', 'squabble', 'snowball', 'become'..."


In [ ]:
Test_x_2=data_test['text_final']
Test_y_2=data_test['category']

In [ ]:
Encoder = LabelEncoder()
Test_y_2 = Encoder.fit_transform(Test_y_2)

In [ ]:
Test_2_X_Tfidf = Tfidf_vect.transform(Test_x_2)

## prediction 1 by naive bayes

In [ ]:
pred_1 = NB.predict(Test_2_X_Tfidf)

In [ ]:
pred_1

array([4, 0, 2, ..., 1, 2, 3])

### accuracy

In [ ]:
print("Naive Bayes Accuracy Score -> ",accuracy_score(pred_1, Test_y_2)*100)

Naive Bayes Accuracy Score ->  98.28751689950428


## prediction using SVM MODEL

In [ ]:
pred_2 = SVM.predict(Test_2_X_Tfidf)

In [ ]:
pred_2

array([4, 0, 2, ..., 1, 2, 3])

In [ ]:
print("SVM Accuracy Score -> ",accuracy_score(pred_2, Test_y_2)*100)

SVM Accuracy Score ->  99.41415051825146
